In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
# from Meanfield_Dyn_util import *


In [ ]:
''' data used 24May'''
# e:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\21Mar_eigenvalues_Gaussian\ConnStats_Gauss_21MarPRX_normlr_N1000_fixedbarJE.npz e:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\21Mar_eigenvalues_Gaussian\ConnStats_Gauss_21MarPRX_normlr_N750_lambda0.npz e:\Dropbox\DailyWork\Allen_project\preparation4paper_Data\21Mar_eigenvalues_Gaussian\ConnStats_Gauss_21MarPRX_normlr_N2000_lambda0.npz

#### helper functions

In [2]:
### dynamics
def iidGaussian(stats,shapem):
	mu,sig = stats[0],stats[1]
	nx,ny = shapem[0],shapem[1]
	return np.random.normal(mu,sig,(nx,ny))

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`
    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.
    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.
    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.
    Returns
    -------
    matplotlib.patches.Ellipse
    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d
def equations(x,data):
    '''
    Function: compute the largest real eigenvalue outlier of the matrix with chain motifs
    eqleft = (outlier**2+gaverage**2*tau)*(outlier**2-gaverage**2*tau*(N-1))
    eqright = outlier*eigvAm[0]*(outlier**2-gaverage**2*tau*(N-1))+gaverage**2*tau*eigvAm[0]*N
    f(outlier) = eqleft - eqright = 0
    params:
    data = [g, tau_chn, lambda_0, N]
    '''
    outlier = x
    gaverage,tau,eigvAm,N = data[0],data[1],data[2],data[3]
    eqleft = (outlier**2+gaverage**2*tau)*(outlier**2-gaverage**2*tau*(N-1))
    eqright = outlier*eigvAm*(outlier**2-gaverage**2*tau*(N-1))+gaverage**2*tau*eigvAm*outlier*N
    # eqright = outlier*eigvAm*(outlier**2+gaverage**2*tau)
    
    return (eqleft-eqright)
#### compute corresponding parameters and variables in the equivalent sparse networks
### Epopulation
def compute_AdjacencyParams(x,JE,JI,sigmae,sigmai,Nparams):
    ce,ci = x[0],x[1]
    NE,NI = Nparams[0],Nparams[1]
    N = NE+NI
    J = JE/NE/ce 
    Jg = JI/NI/ci
    
    sigmae2 = J**2*ce*(1-ce)*N 
    sigmai2 = Jg**2*ci*(1-ci)*N 
    
    resultse = sigmae**2-sigmae2
    resultsi = sigmai**2-sigmai2
    return np.array([resultse,resultsi])

#### Low-rank properties change with the motif statistics

### Network with Gaussian-distributed synaptic weights

In [5]:
JE, JI = 0.65,1.65#current_paradoxical#1.5, 2.0#previous_no_paradoxical
g_max,tau_max = 0.2,0.1 # previous_no_paradoxical#0.15,.2 # previous_no_paradoxical
ntau,trials   = 11,30+6#3,30+6#6, 6
tau_series = np.linspace(0,tau_max,ntau)#np.linspace(0,0.2,ntau)

In [6]:
idnn = 1
nn = [600,800,1000,1200,1400,1600]#1600]
NE, NI = nn[idnn], nn[idnn] 
gamma = 1/4
NI = int(NE*gamma)
N = NE+NI
### recompute JE and JI 
# barJE, barJI = 0.65/800,1.65/200 
# JE, JI = barJE*NE, barJI*NI  
print('new JE and JI:',JE, JI)
Nparams = np.array([NE,NI])
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 


# recordings
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
htau = tau_series[1]-tau_series[0]
### also record the reconstructed eigenvectors
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))


### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1] = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left

new JE and JI: 0.65 1.65


In [35]:
### if you want to re-run the simulations for connectivity, set flag to be True
rerun_conn = True  
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))
### simulation using the low-rank framework
gaverage = g_max
switch = 1
## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2))

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))

intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population
if rerun_conn:
    for ktrial in range(trials):
        #### print the process   
        print('~~~~~~~~~~~trial:',ktrial,'~~~~~~~~~~~~~~~')
        xr      = iidGaussian([0,gaverage/np.sqrt(N)],[N,N])
        chneta  = iidGaussian([0,gaverage/np.sqrt(N)],[N,6])
        xrec    = iidGaussian([0,gaverage/np.sqrt(N)],[N,N])
        ### zscore
        xr   = stats.zscore(xr.flatten())
        xr   = xr*gaverage/np.sqrt(N)
        xr   = np.reshape(xr,(N,N))
        ### zscore
        xrec = stats.zscore(xrec.flatten())
        xrec = xrec*gaverage/np.sqrt(N)
        xrec = np.reshape(xrec,(N,N))
        ### zscore
        chneta[:,0]  = stats.zscore(chneta[:,0])
        chneta[:,0] *=(gaverage/np.sqrt(N))
        ### ---------------------
        intg_ov = np.zeros(2)
        intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
        mean_pre = np.array([JE-JI,0])
        mean_total_change = np.zeros(2)
        z_pre = np.zeros((N,N))
        for it, tau in enumerate(tau_series):
            a    = np.sqrt(tau)
            zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec = a*xrec-a*xrec.copy().T
            # zr   = zrow.copy()+zcol.copy()+np.sqrt(1-4*tau)*xr+gammarec
            zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr
            # zr = xrec*np.sqrt(tau/2)+xrec.copy().T*np.sqrt(tau/2)+np.sqrt(1-tau)*xr ### RECIPROCAL        
            ha    = np.sqrt(htau)
            hzrow = ha*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            hzcol = ha*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            hgammarec = ha*xrec-ha*xrec.copy().T
            # hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-4*htau)*xr+hgammarec
            hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-2*htau)*xr
            DELTA_Z = zr-z_pre
            hzr   = DELTA_Z.copy()

            ### generate J connectivity matrix
            Jchn = Jbar.copy()+zr.copy()     
            eigvchn, eigrvec = la.eig(Jchn)
            eigvchn_,eiglvec = la.eig(Jchn.copy().T)
            assert(np.sum(np.abs(eigvchn[0]-eigvchn_[0]))<1e-9)## check the consistency of left and right eigenvectors
            ### normalization
            reig  = np.squeeze(eigrvec[:,:].copy())
            leig0 = np.squeeze(eiglvec[:,:].copy()) 
            normval = np.sum(reig.copy()*leig0.copy(),axis=0)
            norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
            normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
            leig = leig0.copy()/normval.copy()### left eigenvector normalization
            
            if np.mean(reig[:NE,0])<0:
                reig[:,0]*=-1
                leig[:,0]*=-1
                leig0[:,0]*=-1
            if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
                reig[:,1]*=-1
                leig[:,1]*=-1
                leig0[:,1]*=-1
            ### numerical std 
            intg_std_num_series[ktrial,it,0,0] = np.std(reig[:NE,0])
            intg_std_num_series[ktrial,it,0,1] = np.std(reig[NE:,0])
            intg_std_num_series[ktrial,it,1,0] = np.std(reig[:NE,1])
            intg_std_num_series[ktrial,it,1,1] = np.std(reig[NE:,1])
            
            # ### inverse left eigenvector
            # inveigrvec = la.inv(eigrvec)
            # leig = np.squeeze(inveigrvec[:,:].copy()) # inverse
            # leig = leig.copy().T
            # normval = np.sum(reig.copy()*leig.copy(),axis=0)
            # normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
            # leig = leig.copy()/normval.copy() ## without eigenvalue
            
            eigvchn_series[ktrial,it,:]    = eigvchn.copy()#eigvw_norm.copy()#
            eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
            eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
            eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#

~~~~~~~~~~~trial: 0 ~~~~~~~~~~~~~~~


C:\Users\Administrator\AppData\Local\Temp\ipykernel_94880\470740581.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_94880\470740581.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
C:\Users\Administrator\AppData\Local\Temp\ipykernel_94880\470740581.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
C:\Users\Administrator\AppData\Local\Temp\ipykernel_94880\470740581.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#


~~~~~~~~~~~trial: 1 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 2 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 3 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 4 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 5 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 6 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 7 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 8 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 9 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 10 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 11 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 12 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 13 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 14 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 15 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 16 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 17 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 18 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 19 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 20 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 21 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 22 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 23 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 24 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 25 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 26 ~~~~~~~~~~~~~~~
~~~~~~~~~~~trial: 27 ~~~~~~~~~~~~~~~
~~~~~~~~~~

In [7]:
### compute the theoretical eigenvalue outliers and the spectral radius
### recording 
gaverage = g_max
eigvchn_theo = np.zeros((ntau,2))
radius_theo_map  = np.zeros(ntau)
        
for it, tau in enumerate(tau_series):
    ### solve the equations
    sigmae, sigmai = gaverage, gaverage
    
    x0 = np.array([0.01,0.01])
    sol = fsolve(compute_AdjacencyParams,x0,args=(JE,JI,sigmae,sigmai,Nparams))
    ce,ci = sol[0],sol[1]
    J = JE/NE/ce
    g = JI/NI/ci/J
    print('ce,ci:',ce,ci)
    print('J,g:',J,g)
    ### generate the corresponding sparse matrix
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    tau_rec_vec = np.array([[2*tau,-2*tau],[-2*tau,2*tau]])
    # tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,-tau],[-tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
    ### then the theoretical solutions for the eigenvalue outliers
    ### theoretical solutions
    x0 = np.mean(eigvchn_series[:,it,:2].real,axis=0)
    N = int(NE+NI)
    for ir in range(2):
        t1 = fsolve(equations,x0[ir].real,args=[gaverage,tau,(JE-JI),N])
        eigvchn_theo[it,ir] = t1

ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896
ce,ci: 0.01623594965167502 0.6298438403706231
J,g: 0.05004326925318943 0.26174318832377896

d:\UseApp\Anaconda\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
d:\UseApp\Anaconda\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [8]:
params = {'gaverage':gaverage,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'JE':JE,
          'JI':JI,
          'tau_series':tau_series,
          }

In [19]:
print('parameters:',params)

parameters: {'gaverage': 0.2, 'gamma': 0.25, 'NE': 800, 'NI': 200, 'JE': 0.65, 'JI': 1.6500000000000001, 'tau_series': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])}


In [13]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius


Eigenvalues and spectral radius

In [15]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 4  6 27  9  0  3 19  8 31 28 11 15 30 25 22 13 10 23 29 16  2 35 24 21
 32 12 18 34 14  5] 30


In [24]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,radius_theo_map,'r',label='theoretical')
ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label='numerical')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0)-np.std(radius_num_map,axis=0),np.mean(radius_num_map,axis=0)+np.std(radius_num_map,axis=0),color='gray',alpha=0.3)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,eigvchn_theo[:,0],'r',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'r')
ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
ax.fill_between(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0)-np.std(eigvchn_series[:,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[:,:,0].real,axis=0),color='gray',alpha=0.3)
ax.fill_between(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0)-np.std(eigvchn_series[:,:,1].real,axis=0),np.mean(eigvchn_series[:,:,1].real,axis=0)+np.std(eigvchn_series[:,:,1].real,axis=0),color='gray',alpha=0.3)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
ax.legend()

In [42]:
print('eigvchn numerical:',np.mean(eigvchn_series[:,:,:2],axis=0))
print('eigvchn theoretical:',eigvchn_theo)
print('radius numerical:',np.mean(radius_num_map,axis=0))
print('radius theoretical:',radius_theo_map)

eigvchn numerical: [[-0.99626476+0.j          0.03176039+0.10986565j]
 [-1.23701062+0.j          0.21863476+0.04438241j]
 [-1.41746129+0.j          0.41004728+0.00338007j]
 [-1.56806503+0.j          0.56335762+0.j        ]
 [-1.70000541+0.j          0.69635091+0.j        ]
 [-1.81886104+0.j          0.81572233+0.j        ]
 [-1.92788809+0.j          0.92508915+0.j        ]
 [-2.02918253+0.j          1.02663431+0.j        ]
 [-2.12418728+0.j          1.12183586+0.j        ]
 [-2.21394577+0.j          1.21175514+0.j        ]
 [-2.29924145+0.j          1.29718601+0.j        ]]
eigvchn theoretical: [[-1.00000000e+00  7.30230634e-51]
 [-1.24135012e+00  2.41350120e-01]
 [-1.42152048e+00  4.21520483e-01]
 [-1.57182088e+00  5.71820881e-01]
 [-1.70349491e+00  7.03494911e-01]
 [-1.82211951e+00  8.22119510e-01]
 [-1.93094374e+00  9.30943745e-01]
 [-2.03205744e+00  1.03205744e+00]
 [-2.12689889e+00  1.12689889e+00]
 [-2.21650808e+00  1.21650808e+00]
 [-2.30166590e+00  1.30166590e+00]]
radius numer

In [44]:
params = {'gaverage':gaverage,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'JE':JE,
          'JI':JI,
          'tau_series':tau_series,
          }
lst = [eigvchn_series, eigrvec_series, eiglvec_series,
        eiglvec0_series, norml0_series, params,
        eigvchn_theo,radius_theo_map,
        radius_num_map]
stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
        "eiglvec0_series, norml0_series, params,"
        "eigvchn_theo,radius_theo_map,"
        "radius_num_map"]
data = list_to_dict(lst=lst, string=stg)
# data_name = "/Users/yuxiushao/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/ConnStats_Gauss_2FebPRX_36trials_correction_paradparams.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/21Mar_eigenvalues_Gaussian/ConnStats_Gauss_21MarPRX_normlr_N750_lambda0.npz"
ConnStats_Gauss_21MarPRX_normlr_N1000.npz
# np.savez(data_name, **data)

In [11]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/21Mar_eigenvalues_Gaussian/ConnStats_Gauss_21MarPRX_normlr_N1000_fixedbarJE.npz"
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigvchn_theo = data['eigvchn_theo']
# radius_theo_map = data['radius_theo_map']
# radius_num_map = data['radius_num_map']
params = data['params']
tau_series= params[()]['tau_series']

In [17]:
tau_series

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])

In [12]:
params

array({'gaverage': 0.2, 'gamma': 0.25, 'NE': 800, 'NI': 200, 'JE': 0.65, 'JI': 1.6500000000000001, 'tau_series': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])},
      dtype=object)

In [18]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
# ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',marker='x',label='theoretical')
# ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',marker='x')

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

# ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
# ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
# ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red', ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)
### the color change with gradient 

# ax.scatter(np.mean(eigvchn_series[idx_eff,:,0],axis=0),eigvchn_theo[:,0],cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
# ax.scatter(np.mean(eigvchn_series[idx_eff,:,1],axis=0),eigvchn_theo[:,1],cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')

# ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
ax.set_ylim([-4.5,3.5])
ax.set_yticks([-4,0,3])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [23]:
### scatter plot the predicted eigenvalues eigvchn_theo against the numerical mean np.mean(eigvchn_series[:,:,0],axis=0)
fig, ax = plt.subplots(figsize=(4,4))
### theoretical eigenvlaues eigvchn_theo against the numerical mean np.mean(eigvchn_series,axis=0)
ax.scatter(np.mean(eigvchn_series[idx_eff,:,0],axis=0),eigvchn_theo[:,0],cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
ax.scatter(np.mean(eigvchn_series[idx_eff,:,1],axis=0),eigvchn_theo[:,1],cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')
### refine the figure
# ax.set_xticks([-3,0,3])
# ax.set_yticks([-3,0,3])
# ax.set_xlim([-3,3])
# ax.set_ylim([-3,3])

ax.set_xticks([-2.0,0,1.0])
ax.set_yticks([-2.0,0,1.0])
ax.set_xlim([-2.0,1.0])
ax.set_ylim([-2.0,1.0])
### set x-axis and y-axis to zero
ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
# ax.plot([-3,3],[-3,3],lw=1.5,color='gray',linestyle='--',alpha=0.5)
ax.plot([-2.5,2.5],[-2.5,2.5],lw=1.5,color='gray',linestyle='--',alpha=0.5)
ax.set_xlabel('mean of numerical')
ax.set_ylabel('theoretical prediction')

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\collections.py:192: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Text(0, 0.5, 'theoretical prediction')

loading recorded variables

In [24]:
### loading data and figure out how does the eigenvalue outliers change with the change of Network size N   

### numerical and theoretical eigenvalue outliers
nn = [600,1600]#
itau = -1
eignum_series, eigtheo_series = np.zeros((len(nn),trials-6,2)), np.zeros((len(nn),2))
NNN = (np.array(nn)*5/4).astype(int)
alphass = 1
for idx, netsize in enumerate(NNN):
    if idx == 0:
        continue
    # data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Nscaling_data/GaussianNets/ConnStats_Gauss_16MarPRX_N"+str(netsize)+".npz"
    data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/21Mar_eigenvalues_Gaussian/ConnStats_Gauss_21MarPRX_normlr_N"+str(netsize)+"_lambda0.npz"
    data = np.load(data_name)
    eigvchn_series, eigvchn_theo = data['eigvchn_series'], data['eigvchn_theo']
    ### sort eigvchn_series 
    idx_eff = np.argsort(eigvchn_series[:,-1,0].real)
    idx_eff = idx_eff[3:-3]
    radius_theo_map, radius_num_map = data['radius_theo_map'], data['radius_num_map']
    eignum_series[idx,:,:] = (eigvchn_series[idx_eff,itau,:2])
    eigtheo_series[idx,:] = eigvchn_theo[itau,:]
    print('Netsize:',netsize)
    print('eigvchn_theo:',eigvchn_theo[itau,:])
    ax.plot(tau_series,eigvchn_theo[:,0],'darkred',label='theoretical',alpha=alphass,ls='--')
    ax.plot(tau_series,eigvchn_theo[:,1],'darkgreen',alpha=alphass,ls='--')
    # ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
    # ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
    # ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
    ### plot error bar  
    ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='darkred',ecolor='darkred',alpha=alphass,ls='',elinewidth=1.5)
    # ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
    ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='darkgreen',ecolor='darkgreen',alpha=alphass,ls='',elinewidth=1.5)
ax.plot(tau_series,np.ones(len(tau_series)),'k',linewidth=1.5)    

Netsize: 2000
eigvchn_theo: [-3.37158493  2.37158493]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_2120\3374709095.py:20: ComplexWarning: Casting complex values to real discards the imaginary part
  eignum_series[idx,:,:] = (eigvchn_series[idx_eff,itau,:2])


In [87]:
### plot the barplot of the eigenvalue outliers, x axis is the network size
fig,ax = plt.subplots(figsize=(4,2))
ax.bar(np.arange(len(nn))-0.2,np.mean(eignum_series[:,:,0],axis=1),width=0.4,color='tab:red',label='numerical',alpha=0.25)
ax.bar(np.arange(len(nn))+0.2,np.mean(eignum_series[:,:,1],axis=1),width=0.4,color='tab:green',label='numerical',alpha=0.25)
ax.errorbar(np.arange(len(nn))-0.2,np.mean(eignum_series[:,:,0],axis=1),yerr=np.std(eignum_series[:,:,0],axis=1),fmt='o',color='k',label='std')
ax.errorbar(np.arange(len(nn))+0.2,np.mean(eignum_series[:,:,1],axis=1),yerr=np.std(eignum_series[:,:,1],axis=1),fmt='o',color='k',label='std')
### plot the theoretical eigenvalue outliers
ax.plot(np.arange(len(nn))-0.2,eigtheo_series[:,0],'tab:red',label='theoretical')
ax.plot(np.arange(len(nn))+0.2,eigtheo_series[:,1],'tab:green',label='theoretical')
### move the x axis to the center 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
### set the xticks and xlabels
ax.set_xticks(np.arange(len(nn)))
ax.set_xticklabels(NNN)
ax.set_xlabel('Network size')
ax.set_ylabel('Eigenvalue outliers')
ax.legend()
### set ylim and yticks
ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
plt.show()


In [11]:
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Nscaling_data/GaussianNets/ConnStats_Gauss_16MarPRX_N750_standard.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Nscaling_data/GaussianNets/ConnStats_Gauss_16MarPRX_N1000_fixedbarJE.npz"
# np.savez(data_name, **data)
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
# eigrvec_series_rec = data['eigrvec_series_rec']
# eiglvec_series_rec = data['eiglvec_series_rec']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
# intg_mean_series = data['intg_mean_series']
# leig0mean_series = data['leig0mean_series']
params = data['params']


In [12]:
params

array({'gaverage': 0.1, 'gamma': 0.25, 'NE': 800, 'NI': 200, 'JE': 0.65, 'JI': 1.6500000000000001, 'tau_series': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])},
      dtype=object)

In [13]:
# ### also loading other variables 
# eigrvec_series = data['eigrvec_series']
# eiglvec_series = data['eiglvec_series']
# eigrvec_series_rec = data['eigrvec_series_rec']
# eiglvec_series_rec = data['eiglvec_series_rec']
# eiglvec0_series = data['eiglvec0_series']
# norml0_series = data['norml0_series']
# params = data['params']
# intg_ov_series = data['intg_ov_series']
# first_perturb_ov = data['first_perturb_ov']
# first_perturb_ovP = data['first_perturb_ovP']
# intg_mean_series = data['intg_mean_series']
# intg_std_series = data['intg_std_series']
# intg_std_num_series = data['intg_std_num_series']
# mean_shift_ov = data['mean_shift_ov']
# intg_crossov_series = data['intg_crossov_series']
# intg_crossovPop_series = data['intg_crossovPop_series']

Mean of the left and right eigenvectors 

In [281]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']

mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 10
ths = 0.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)

            mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 3
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,0].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,1].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        
        

In [282]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,0],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$E$')
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),color='black',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,0],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),color='gray',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
# ax[0].set_ylim(-1,1)
# ax[0].set_yticks([-1,1])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population

# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,1],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),color='orange',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,1],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[idx_eff,:,1,1]*eigvchn_series[idx_eff,:,1],axis=0),color='green',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-0.4,0.02)
# ax[1].set_yticks([-0.4,0.])
ax[1].set_ylim(-0.35,0.02)
ax[1].set_yticks([-0.35,0.])
# ax[1].set_ylim(-1.35,1.2)
# ax[1].set_yticks([-1.35,1.])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5398: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[0] = start
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5399: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N + 1] = end
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5402: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5404: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N+2:, 1] = dep2slice[::-1]
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [283]:
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=1.0)
# for i in range(trials):
#     ax[0].scatter(tau_series,mean_reigvec_num_series[i,:,0,0])
#     print(mean_reigvec_num_series[i,:,0,0])
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
## second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1,0.2])
ax[0].set_xlim(0,tau_max+0.02/8)
ax[0].set_ylim(-0.01,0.04)
ax[0].set_yticks([0,0.04])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)

### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1,0.2])
ax[1].set_xlim(0-dtau/8.0,tau_max+dtau/8)
ax[1].set_ylim(-0.01,0.04)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
fig.tight_layout()

In [284]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,0],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$E$')
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),color='black',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,0],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),color='gray',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)
## excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
ax[0].set_ylim(-0.1,0.3)
ax[0].set_yticks([-0.1,0.3])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population

# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,1],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),color='orange',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,1],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[idx_eff,:,1,1]*eigvchn_series[idx_eff,:,1],axis=0),color='green',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)
## move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.4,0.02)
ax[1].set_yticks([-0.4,0.])
ax[1].set_ylim(-1,0.5)
ax[1].set_yticks([-1,0.5])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [78]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [286]:
params = {'gaverage':gaverage,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'JE':JE,
          'JI':JI,
          'tau_series':tau_series,
          }
lst = [eigvchn_series, eigrvec_series, eiglvec_series,
       eigrvec_series_rec, eiglvec_series_rec,
        eiglvec0_series, norml0_series, params,
        intg_mean_series,intg_std_series,intg_std_num_series,mean_shift_ov,
        intg_crossov_series,intg_crossovPop_series,leig0mean_series]
stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
       "eigrvec_series_rec, eiglvec_series_rec,"
        "eiglvec0_series, norml0_series, params,"
        "intg_mean_series,intg_std_series,intg_std_num_series,mean_shift_ov,"
        "intg_crossov_series,intg_crossovPop_series,leig0mean_series"]
data = list_to_dict(lst=lst, string=stg)
# data_name = "/Users/yuxiushao/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/ConnStats_Gauss_2FebPRX_36trials_correction_paradparams.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/ConnStats_Gauss_12MarPRX_36trials_N1000_standard.npz"
# np.savez(data_name, **data)

Compare the reconstructed left eigenvectors with the numerical one

In [287]:
ktrial = 15
its = [4,ntau-1]
fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
for i,it in enumerate(its):
    ### reconstructed left eigenvectors v.s. numerical left eigenvectors
    ax[i].scatter(eiglvec_series_rec[ktrial,it,:,0],eiglvec_series[ktrial,it,:,0]*eigvchn_series[ktrial,it,0],color='black',s=10,label='rank 1')
    ax[i].scatter(eiglvec_series_rec[ktrial,it,:,1],eiglvec_series[ktrial,it,:,1]*eigvchn_series[ktrial,it,1],color='gray',s=10,label='rank 2')
    ax[i].plot([-1,1],[-1,1],color='black',linestyle='--')
    ax[i].set_xlabel('Reconstructed left eigenvectors',fontsize=14)
    ax[i].set_ylabel('Numerical left eigenvectors',fontsize=14)
    ax[i].set_title(r'$\tau=$'+str(tau_series[it]),fontsize=14)
    ax[i].set_aspect('equal')
    ax[i].set_xlim(-.5,.5)
    ax[i].set_ylim(-.5,.5)
    ax[i].set_xticks([-0.5,0,0.5])
    ax[i].set_yticks([-0.5,0,0.5])
fig.tight_layout()

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\collections.py:192: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


In [288]:
### same as the above but for the right eigenvectors
fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
for i,it in enumerate(its):
    ### reconstructed right eigenvectors v.s. numerical right eigenvectors
    ax[i].scatter(eigrvec_series_rec[ktrial,it,:,0],eigrvec_series[ktrial,it,:,0],color='black',s=10,label='rank 1')
    ax[i].scatter(eigrvec_series_rec[ktrial,it,:,1],eigrvec_series[ktrial,it,:,1],color='gray',s=10,label='rank 2')
    ax[i].plot([-1,1],[-1,1],color='black',linestyle='--')
    ax[i].set_xlabel('Reconstructed right eigenvectors',fontsize=14)
    ax[i].set_ylabel('Numerical right eigenvectors',fontsize=14)
    ax[i].set_title(r'$\tau=$'+str(tau_series[it]),fontsize=14)
    ax[i].set_aspect('equal')
    ax[i].set_xlim(-.15,.15)
    ax[i].set_ylim(-.15,.15)
    ax[i].set_xticks([-0.1,0,0.1])
    ax[i].set_yticks([-0.1,0,0.1])
fig.tight_layout()   

Scatter ploting the random components on the left and right eigenvectors, showing that the random component are positively/negatively correlated

In [289]:
### plot random components on left and right eigenvectors when tau == 0.02 and ktrial = 15
ktrial = idx_eff[6]
its = [int((ntau-1)*2/3),ntau-1]
fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
for i,it in enumerate(its):
    ax[i].scatter(eigrvec_series[ktrial,it,:NE,0],eigrvec_series[ktrial,it,:NE,1],color='tab:red',s=10,label='right eigenvector '+r'$E$',alpha=0.25)
    ax[i].scatter(eigrvec_series[ktrial,it,NE:,0],eigrvec_series[ktrial,it,NE:,1],color='tab:blue',s=10,label='right eigenvector '+r'$I$',alpha=0.25)
    ### equal axis
    ax[i].set_aspect('equal')
    ax[i].set_xlim(-0.15,0.15)
    ax[i].set_ylim(-0.15,0.15)
    ax[i].set_xlabel('Random component rank 1',fontsize=14)
    ax[i].set_ylabel('Random component rank 2',fontsize=14)
    ax[i].legend()
    ax[i].set_title('Right eigenvector '+r'$\tau=$'+str(tau_series[it]),fontsize=14)
    ax[i].set_xticks([-0.1,0,0.1])
    ax[i].set_yticks([-0.1,0,0.1])
    ### remove the top and right axes
    ax[i].spines['top'].set_color('none')
    ax[i].spines['right'].set_color('none')
    ### move the x-axis and y-axis to the center
    ax[i].spines['bottom'].set_position(('data',0))
    ax[i].spines['left'].set_position(('data',0))
fig.tight_layout()


In [290]:
# ktrial = 8
### compute the slope of the right eigenvectors (correlated, rank-1 and rank-2)
### using linear regression and the slope is the first element of the output
from scipy.stats import linregress
slope_eigrvec = np.zeros((len(idx_eff),ntau,2))
slope_eigrvec_rec = np.zeros((len(idx_eff),ntau,2))
for ik,ktrial in enumerate(idx_eff):
    for it in range(0,ntau):
        slope_eigrvec[ik,it,0] = linregress(eigrvec_series[ktrial,it,:NE,0],eigrvec_series[ktrial,it,:NE,1])[0]
        slope_eigrvec_rec[ik,it,0] = linregress(eigrvec_series_rec[ktrial,it,:NE,0],eigrvec_series_rec[ktrial,it,:NE,1])[0]
        slope_eigrvec[ik,it,1] = linregress(eigrvec_series[ktrial,it,NE:,0],eigrvec_series[ktrial,it,NE:,1])[0]
        slope_eigrvec_rec[ik,it,1] = linregress(eigrvec_series_rec[ktrial,it,NE:,0],eigrvec_series_rec[ktrial,it,NE:,1])[0]
        
        if np.abs(slope_eigrvec[ik,it,0])>2:
            slope_eigrvec[ik,it,0] = np.nan
        if np.abs(slope_eigrvec[ik,it,1])>2:
            slope_eigrvec[ik,it,1] = np.nan
        if np.abs(slope_eigrvec_rec[ik,it,0])>2:
            slope_eigrvec_rec[ik,it,0] = np.nan
        if np.abs(slope_eigrvec_rec[ik,it,1])>2:
            slope_eigrvec_rec[ik,it,1] = np.nan
            
### plot the (trial-mean) slope of the right eigenvectors, change with tau
fig, ax = plt.subplots(figsize=(4,2))
ax.plot(tau_series,np.nanmean(slope_eigrvec[:,:,0],axis=0),color='black',linewidth=1.5,linestyle='--',label='numerical',marker='o')
ax.plot(tau_series,np.nanmean(slope_eigrvec_rec[:,:,0],axis=0),color='black',linewidth=1.5,label='reconstructed',marker='o')

ax.set_xlim(tau_series[0]-dtau/8,tau_series[its[1]]+dtau/8)
# ax.set_ylim(-1.5,0.1)
# ax.set_yticks([-1,0])
ax.set_ylim(-2.5,0.1)
ax.set_yticks([-2,0])
ax.set_xticks([tau_series[0],tau_series[its[1]]])
### move the x-axis to  the cecnter
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
ax.set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax.set_ylabel('Slope of \nright eigenvector',fontsize=12)
ax.legend()
# fig.tight_layout()

In [291]:
#### same as above but for left eigenvectors
# xlims = [-1.5,1.5]
# ylims = [-1.5,1.5]
xlims = [-.5,.5]
ylims = [-.5,.5]
fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
for i,it in enumerate([2,10]):#its):
    ax[i].scatter(eiglvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0],eiglvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1],color='tab:red',s=10,label='left eigenvector '+r'$E$')
    ax[i].scatter(eiglvec_series[ktrial,it,NE:,0]*eigvchn_series[ktrial,it,0],eiglvec_series[ktrial,it,NE:,1]*eigvchn_series[ktrial,it,1],color='tab:blue',s=10,label='left eigenvector '+r'$I$')
    
    # ax[i].scatter(eiglvec_series_rec[ktrial,it,:NE,0],eiglvec_series_rec[ktrial,it,:NE,1],color='tab:red',s=10,label='left eigenvector '+r'$E$',alpha=0.25)
    # ax[i].scatter(eiglvec_series_rec[ktrial,it,NE:,0],eiglvec_series_rec[ktrial,it,NE:,1],color='tab:blue',s=10,label='left eigenvector '+r'$I$',alpha=0.25)
    ### equal axis
    ax[i].set_aspect('equal')
    ax[i].set_xlim(xlims)
    ax[i].set_ylim(ylims)
    ax[i].set_xlabel('Random component rank 1',fontsize=14)
    ax[i].set_ylabel('Random component rank 2',fontsize=14)
    ax[i].legend()
    ### remove the top and right axes
    ax[i].spines['top'].set_color('none')
    ax[i].spines['right'].set_color('none')
    ### move the x-axis and y-axis to the center
    ax[i].spines['bottom'].set_position(('data',0))
    ax[i].spines['left'].set_position(('data',0))
    ax[i].set_title('Left eigenvector '+r'$\tau=$'+str(tau_series[it]),fontsize=14)
    ax[i].set_xticks([xlims[0],0,xlims[-1]])
    ax[i].set_yticks([xlims[0],0,xlims[-1]])
fig.tight_layout()


In [292]:
dtau = tau_series[1]-tau_series[0]
slope_eiglvec     = np.zeros((len(idx_eff),ntau,2)) ### population
slope_eiglvec_rec = np.zeros((len(idx_eff),ntau,2))  ### population
for ik, ktrial in enumerate(idx_eff):
    for it in range(0,ntau):
        slope_eiglvec[ik,it,0]     = linregress(eiglvec_series[ktrial,it,:NE,0]*eigvchn_series[ktrial,it,0].real,eiglvec_series[ktrial,it,:NE,1]*eigvchn_series[ktrial,it,1].real)[0]
        slope_eiglvec_rec[ik,it,0] = linregress(eiglvec_series_rec[ktrial,it,:NE,0],eiglvec_series_rec[ktrial,it,:NE,1])[0]
        
        slope_eiglvec[ik,it,1]     = linregress(eiglvec_series[ktrial,it,NE:,0]*eigvchn_series[ktrial,it,0].real,eiglvec_series[ktrial,it,NE:,1]*eigvchn_series[ktrial,it,1].real)[0]
        slope_eiglvec_rec[ik,it,1] = linregress(eiglvec_series_rec[ktrial,it,NE:,0],eiglvec_series_rec[ktrial,it,NE:,1])[0]
        
        if np.abs(slope_eiglvec[ik,it,0])>2:
            slope_eiglvec[ik,it,0] = np.nan
        if np.abs(slope_eiglvec[ik,it,1])>2:
            slope_eiglvec[ik,it,1] = np.nan
        if np.abs(slope_eiglvec_rec[ik,it,0])>2:
            slope_eiglvec_rec[ik,it,0] = np.nan
        if np.abs(slope_eiglvec_rec[ik,it,1])>2:
            slope_eiglvec_rec[ik,it,1] = np.nan
            
### plot the (trial-mean) slope of the right eigenvectors, change with tau
fig, ax = plt.subplots(figsize=(4,2))
### PLOT THE MEAN AND VARIANCE OF SLOPE_EIGLVEC[:,:,0]
meaneiglvecE, stdeiglvecE = np.nanmean(slope_eiglvec[:,:,0],axis=0),np.nanstd(slope_eiglvec[:,:,0],axis=0)
# ax.fill_between(tau_series,meaneiglvecE-stdeiglvecE,meaneiglvecE+stdeiglvecE,facecolor='gray',alpha=0.3)
ax.plot(tau_series,np.nanmean(slope_eiglvec[:,:,0],axis=0),color='black',linewidth=1.5,linestyle='--',label='numerical',marker='o')
ax.plot(tau_series,np.nanmean(slope_eiglvec_rec[:,:,0],axis=0),color='black',linewidth=1.5,label='reconstructed',marker='o')

# ax.plot(tau_series,np.nanmean(slope_eiglvec[:,:,1],axis=0),color='gray',linewidth=1.5,linestyle='--',label='numerical')
# ax.plot(tau_series,np.nanmean(slope_eiglvec_rec[:,:,1],axis=0),color='gray',linewidth=1.5,label='reconstructed')

ax.set_xlim(tau_series[0]-dtau/8,tau_series[its[1]]+dtau/8)
ax.set_ylim(-0.5,1.5)
ax.set_yticks([0,1])
ax.set_xticks([tau_series[0],tau_series[its[1]]])
### move the x-axis to  the cecnter
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',tau_series[0]))
ax.set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax.set_ylabel('Slope of \nleft eigenvector',fontsize=12)
ax.legend()
# fig.tight_layout()

In [28]:
plt.close('all')

## dynamics

In [160]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@(x)+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)

#### Network dynamics in response to the uniform input

We use linear response theory here, and focus on the population-averaged responses, for a specific population $p$, the equilibrium firing rate is computed analytically

$$ \langle r\rangle_{i\in N_p}=I^{s,p}+\sum_{r=1}^{R}\frac{a_{m_r}^p}{1-\lambda_r}\sum_{q=E,I}N_qa_{n_r}^qI^{s,q}$$

In [161]:
nn = [50,100,150,250,750,500,800]
NE, NI = nn[-1], nn[-1] 
gamma = 1/4
NI = int(NE*gamma)
N = NE+NI
JE, JI =  0.65, 1.65#1.5, 2.0#1.5, 2.0
Nparams = np.array([NE,NI])
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 

tt  = np.linspace(0,100,1000)
g_max,tau_max = 0.10,.1
ntau = 6### even quit sparse taus can generate accurate predictions
ntau = 11## denser taus
tau_series = np.linspace(0,tau_max,ntau)#np.linspace(0,0.2,ntau)
trials = 30+6 #is the minimal number of iterations
# recordings
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
htau    = tau_series[1]-tau_series[0]

### also record the reconstructed eigenvectors
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))

### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0] = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]     = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct  = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left

Mean network's dynamics in response to uniform stimuli

In [162]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
tt    = np.linspace(0,100,1000)
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [ ]:

### if you want to re-run the simulation
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2))

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))

intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population

gaverage = g_max ### set the magnitude of the connectivity fluctuation
switch = 1

### if you want to re-run the simulations for dynamics, set rerun_dyns to be True 
rerun_dyns = True   
if rerun_dyns:
    for ktrial in range(trials):
        xr      = iidGaussian([0,gaverage/np.sqrt(N)],[N,N])
        chneta  = iidGaussian([0,gaverage/np.sqrt(N)],[N,6])
        xrec    = iidGaussian([0,gaverage/np.sqrt(N)],[N,N])
        ### zscore
        xr   = stats.zscore(xr.flatten())
        xr   = xr*gaverage/np.sqrt(N)
        xr   = np.reshape(xr,(N,N))
        ### zscore
        xrec = stats.zscore(xrec.flatten())
        xrec = xrec*gaverage/np.sqrt(N)
        xrec = np.reshape(xrec,(N,N))
        ### zscore
        chneta[:,0]  = stats.zscore(chneta[:,0])
        chneta[:,0] *=(gaverage/np.sqrt(N))
        ### ---------------------
        intg_ov = np.zeros(2)
        intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
        mean_pre = np.array([JE-JI,0])
        mean_total_change = np.zeros(2)
        z_pre = np.zeros((N,N))
        for it, tau in enumerate(tau_series):
            a    = np.sqrt(tau)
            zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec = a*xrec-a*xrec.copy().T
            # zr   = zrow.copy()+zcol.copy()+np.sqrt(1-4*tau)*xr+gammarec
            zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr
            # zr = xrec*np.sqrt(tau/2)+xrec.copy().T*np.sqrt(tau/2)+np.sqrt(1-tau)*xr ### RECIPROCAL        
            ha    = np.sqrt(htau)
            hzrow = ha*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            hzcol = ha*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            hgammarec = ha*xrec-ha*xrec.copy().T
            # hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-4*htau)*xr+hgammarec
            hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-2*htau)*xr
            DELTA_Z = zr-z_pre
            hzr   = DELTA_Z.copy()

            ### generate J connectivity matrix
            Jchn = Jbar.copy()+zr.copy()
            ### full rank simulation
            xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
            xc_temporal = odesimulation(tt, xinit, Jchn, Inp)
            firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
            
            eigvchn, eigrvec = la.eig(Jchn)
            eigvchn_,eiglvec = la.eig(Jchn.copy().T)
            assert(np.sum(np.abs(eigvchn[0]-eigvchn_[0]))<1e-9)## check the consistency of left and right eigenvectors
            ### normalization
            reig  = np.squeeze(eigrvec[:,:].copy())
            leig0 = np.squeeze(eiglvec[:,:].copy()) 
            normval = np.sum(reig.copy()*leig0.copy(),axis=0)
            norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
            normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
            leig = leig0.copy()/normval.copy()### left eigenvector normalization

            if np.mean(reig[:NE,0])<0:
                reig[:,0]*=-1
                leig[:,0]*=-1
                leig0[:,0]*=-1
            if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
                reig[:,1]*=-1
                leig[:,1]*=-1
                leig0[:,1]*=-1
                
            ### numerical low-rank approximation 
            ov_inp_lowrank,ov_inp_lowrank_div= np.zeros(2),np.zeros(2)
            vec_lowrank_contribution = np.zeros((N,2))
            ### linear response theory approximation 
            for i in range(2):
                ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
                ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
            Equilibrium_lowrank_outliers = Inp.copy()
            for i in range(2):
                vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
                Equilibrium_lowrank_outliers += vec_lowrank_contribution[:,i].copy()
                #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
                contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
                contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
            lowrank_eq_num[ktrial,it,:] = Equilibrium_lowrank_outliers.copy()
            ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 

            
            eigvchn_series[ktrial,it,:]    = eigvchn.copy()#eigvw_norm.copy()#
            eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
            eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
            eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
            # chain_mtx = np.ones((N,N))-np.eye(N)
            # chain_mtx  = gaverage**2*chain_mtx
            chain_mtx = hzr@hzr/htau
            chainE_mtx = gaverage**2*(np.ones((N,N))-np.eye(N))*NE/N #### ERROR!!!! (CHAIN_MTX)xxxx
            chainI_mtx = gaverage**2*(np.ones((N,N))-np.eye(N))*NI/N
            if it<1:
                lvec, rvec = np.squeeze(leigvec0[:,:2]),np.squeeze(reigvec0[:,:2])
                # eigeng     = np.squeeze(eigvchn_series[ktrial,it,:2].copy())
                eigeng     = np.array([JE-JI,0])
                lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
            else:
                lvec, rvec = np.squeeze(eiglvec0_series[ktrial,it-1,:,:2]),np.squeeze(eigrvec_series[ktrial,it-1,:,:2]) ### use the previous eigenvector as the initial condition   
                eigeng = np.squeeze(eigvchn_series[ktrial,it-1,:2].copy())
                ### conditioned mean  
                lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
                lvec_mean[:NE,:],lvec_mean[NE:,:]=np.mean(lvec_mean[:NE,:],axis=0),np.mean(lvec_mean[NE:,:],axis=0)
                rvec_mean[:NE,:],rvec_mean[NE:,:]=np.mean(rvec_mean[:NE,:],axis=0),np.mean(rvec_mean[NE:,:],axis=0)
                
            ### get the appropriate normalization factor
            norm_for_lvec, norm_for_rvec = np.zeros(2),np.zeros(2)
            if it==0:
                hzr_u = xr.copy()
                with_chn = 0
            else:
                hzr_u = hzr.copy()
                with_chn = 1
            
            norm_rvec_temp = np.zeros((N,2))
            norm_lvec_temp = np.zeros((N,2))
            for i in range(2):
                rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
                lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
                ''' Mean Connectivity '''
                current_eigv = eigvchn_series[ktrial,it,i].copy()
                if it==0:
                    intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                    intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                    leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                    norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                    norm_lvec_temp[:,i] = np.squeeze(leig[:,i])
                elif it<3 and i==1: ### 3 is 2 blocks after radius<outlier
                    ### original values are obtained numerically
                    intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                    intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                    leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                    
                    norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                    norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
                else:
                    # if it<=switch:
                    #     eigenvalue_u = np.real(eigeng[i])
                    #     eigenvalue_um = current_eigv.real#np.real(current_eigv)
                    # else:
                    #     eigenvalue_u = np.real(current_eigv)
                    #     eigenvalue_um = np.real(eigeng[i])  
                        
                    eigenvalue_u = np.real(current_eigv)
                    eigenvalue_um = np.real(eigeng[i])  
                        
                    ### norm_for_rvec and norm_for_lvec 
                    rmean_tmp = np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*htau*(chain_mtx@np.reshape(rvec_mean[:,i],(-1,1)))/eigenvalue_um**2,(N,1)) ### mean-shifting 
                    rvec_n[:NE,0] = rvec_n[:NE,0] - np.mean(rvec_n[:NE,0])+rmean_tmp[:NE,0]
                    rvec_n[NE:,0] = rvec_n[NE:,0] - np.mean(rvec_n[NE:,0])+rmean_tmp[NE:,0]
                    norm_for_rvec[i] = la.norm(rvec_n)
                    lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*htau*np.reshape(lvec_mean[:,i],(1,-1))@chain_mtx/np.real(eigeng[i])**2,(N,1))### mean-shifting 
                    lvec_n[:NE,0] = lvec_n[:NE,0] - np.mean(lvec_n[:NE,0])+lmean_tmp[:NE,0]
                    lvec_n[NE:,0] = lvec_n[NE:,0] - np.mean(lvec_n[NE:,0])+lmean_tmp[NE:,0]
                    
                    norm_for_rvec[i] = la.norm(rvec_n) ### normalization factor 
                    norm_rvec_n = np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i] 
                    norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
                    norm_for_lvec[i] = la.norm(lvec_n) ### normalization factor    
                    norm_lvec_n = np.reshape(lvec_n.copy(),(-1,1))/norm_for_lvec[i] 
                    ### need to be re-normalized
                    norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())/norml0_series[ktrial,it,i]
                    norm_lvec_temp[:,i] *= eigvchn_series[ktrial,it,i].real
                    
                    norml = norm_for_lvec[i]
                    ### more simplified version using lvec_mean and rvec_mean
                    intg_ml[:,i] = lmean_tmp[:,0]/norml0_series[ktrial,it,i]*eigenvalue_u/norml#np.real(current_eigv)
                    
                    leig0mean_series[ktrial,it,:,i] =lmean_tmp[:,0]/norml/norml0_series[ktrial,it,i]#*eigenvalue_u#
                    
                    # intg_ml[:,i] = np.squeeze(np.reshape(lvec_mean[:,i].copy(),(-1,1)))/norml0_series[ktrial,it,i]*eigenvalue_u/norml#np.real(current_eigv)
                    
                    # leig0mean_series[ktrial,it,:,i] =np.squeeze(np.reshape(lvec_mean[:,i].copy(),(-1,1)) )/norml/norml0_series[ktrial,it,i]#*eigenvalue_u#                  
                    
                    leig0pre_series[ktrial,it,:,i] =np.reshape(with_chn*htau*np.reshape(lvec_mean[:,i],(1,-1))@chain_mtx/eigeng[i].real**2,(N,1))[:,0]#*eigenvalue_u#
                        
                    intg_mr[:,i] = rmean_tmp[:,0]/norm_for_rvec[i]
                
                norm_rvec_temp[:NE,i] -= np.mean(norm_rvec_temp[:NE,i])
                intg_std_series[ktrial,it,i,0] = np.std(norm_rvec_temp[:NE,i].copy())
                norm_rvec_temp[NE:,i] -= np.mean(norm_rvec_temp[NE:,i])
                intg_std_series[ktrial,it,i,1] = np.std(norm_rvec_temp[NE:,i].copy())
                norm_rvec_temp[:NE,i] += intg_mr[:NE,i]
                norm_rvec_temp[NE:,i] += intg_mr[NE:,i]         
                ### and then the norm_lvec_temp
                norm_lvec_temp[:NE,i] -= np.mean(norm_lvec_temp[:NE,i])
                norm_lvec_temp[NE:,i] -= np.mean(norm_lvec_temp[NE:,i])
                norm_lvec_temp[:NE,i] += intg_ml[:NE,i]
                norm_lvec_temp[NE:,i] += intg_ml[NE:,i]
                ### also record the reconstructed eigenvectors
                eigrvec_series_rec[ktrial,it,:,i] = norm_rvec_temp[:,i].copy()
                eiglvec_series_rec[ktrial,it,:,i] = norm_lvec_temp[:,i].copy()
                
                intg_mean_series[ktrial,it,:,i,0] = np.reshape(intg_ml[:,i],(N,))
                intg_mean_series[ktrial,it,:,i,1] = np.reshape(intg_mr[:,i],(N,))
            
                '''Overall overlap '''
                if it>0:
                    if it<=switch:
                        eigenvalues_use = eigvchn_series[ktrial,it-1,:2].copy()
                        eigenvalues_usem = eigvchn_series[ktrial,it,:2].copy()
                    else:
                        eigenvalues_use = eigvchn_series[ktrial,it,:2].copy()
                        eigenvalues_usem = eigvchn_series[ktrial,it-1,:2].copy()
                    intg_ov[i] = first_perturb_ov[ktrial,it-1,i,i] + with_chn*htau*lvec_mean[:,i].T@chain_mtx@rvec_mean[:,i]/np.real(eigeng[i])/np.real(eigenvalues_usem[i])/norml0_series[ktrial,it,i]*np.real(eigenvalues_use[i])/norm_for_lvec[i]/norm_for_rvec[i]
                else:
                    intg_ov[i] = htau*with_chn*lvec_mean[:,i].T@chain_mtx@rvec_mean[:,i]/np.real(eigeng[i])**2/norml0_series[ktrial,it,i]*eigvchn_series[ktrial,it,i].real/norm_for_lvec[i]/norm_for_rvec[i]
                    
                intg_ov_series[ktrial,it,i] = np.squeeze(intg_ov[i]).real
            
            z_pre= zr.copy()
            ### theoretically compute the equilibrium population-averaged firing rate
            an = np.zeros((2,2),dtype=complex)
            am = np.zeros((2,2),dtype=complex) ## population X rank    
            for ir in range(2):
                an[0,ir] = np.mean(intg_ml[:NE,ir])
                an[1,ir] = np.mean(intg_ml[NE:,ir])
                am[0,ir] = np.mean(intg_mr[:NE,ir])
                am[1,ir] = np.mean(intg_mr[NE:,ir])
                
            ### overlap sum
            overlap_inp = np.zeros(2,dtype=complex) 
            for ir in range(2):
                overlap_inp[ir] = (NE*an[0,ir]*Inp[0]+NI*an[1,ir]*Inp[-1])
                ovs_inplr[ktrial,it,ir] = overlap_inp[ir] ### recording_theory
                overlap_inp[ir]/= (1.0-eigvchn[ir])
                ovs_inplr_div[ktrial,it,ir] = overlap_inp[ir] ### recording theory
            eq_fr = np.zeros(2)
            eq_fr[0],eq_fr[1] = Inp[0],Inp[-1]
            for ir in range(2): ### rank two 
                ### record 
                contributions_lr[ktrial,it,ir,0] = overlap_inp[ir]*am[0,ir] ### excitatory population
                contributions_lr[ktrial,it,ir,1] = overlap_inp[ir]*am[1,ir] ### inhibitory population
                eq_fr[0] += am[0,ir]*overlap_inp[ir]
                eq_fr[1] += am[1,ir]*overlap_inp[ir]
            lowrank_eq[ktrial,it,:] = eq_fr.copy() 
            
            ###
            if it>=1:
                if it<=switch:
                    eigenvalues_use = eigvchn_series[ktrial,it-1,:2].copy()
                    eigenvalues_usem = eigvchn_series[ktrial,it,:2].copy()
                else:
                    eigenvalues_use = eigvchn_series[ktrial,it,:2].copy()
                    eigenvalues_usem = eigvchn_series[ktrial,it-1,:2].copy()
                intg_ovP = first_perturb_ovP[ktrial,it-1,:,:,:].copy()
                for ir in range(2):
                    for jl in range(2):
                        intg_ovP[ir,jl,0] += with_chn*htau*lvec_mean[:,jl].T@(chainE_mtx)@rvec_mean[:,ir]/norm_for_lvec[jl]/norm_for_rvec[ir]/norml0_series[ktrial,it,jl]*eigenvalues_use[jl].real/np.real(eigeng[jl])/np.real(eigenvalues_usem[ir])### current eigenvalues are multiplied
                        intg_ovP[ir,jl,1] += with_chn*htau*lvec_mean[:,jl].T@(chainI_mtx)@rvec_mean[:,ir]/norm_for_lvec[jl]/norm_for_rvec[ir]/norml0_series[ktrial,it,jl]*eigenvalues_use[jl].real/np.real(eigeng[jl])/np.real(eigenvalues_usem[ir])### current eigenvalues are multiplied                  
                        intg_crossovPop_series[ktrial,it,ir,jl,0] = np.squeeze(intg_ovP[ir,jl,0]).real
                        intg_crossovPop_series[ktrial,it,ir,jl,1] = np.squeeze(intg_ovP[ir,jl,1]).real
            
            ### mean shift                    
            meanr, meanl = np.zeros((2,2)),np.zeros((2,2))## rank pop
            meanr[:,0], meanr[:,1] = np.mean(reig[:NE,:2],axis=0),np.mean(reig[NE:,:2],axis=0)
            meanl[:,0], meanl[:,1] = np.mean(leig[:NE,:2],axis=0),np.mean(leig[NE:,:2],axis=0)
            mean_curr, mean_change = np.zeros(2),np.zeros(2)
            for i in range(2):
                mean_curr[i] = (meanr[i,0]*meanl[i,0]*NE+meanr[i,1]*meanl[i,1]*NI)*np.real(eigeng[i])
                mean_change[i] = mean_curr[i] - mean_pre[i]
            
            mean_total_change += with_chn*mean_change ### the first one without any change
            if np.abs(mean_total_change[0])>1000:
                print('too large change')
            if it==1:
                mean_total_change[1] = 0
                mean_change[1] = 0
            mean_shift_ov[ktrial,it,:] = mean_total_change
            mean_pre    = mean_curr
            ##### compute current overlap (based on current mean connectivity)
            noiser, noisel = np.zeros((N,2)),np.zeros((N,2))
            for i in range(2):
                noiser[:NE,i], noiser[NE:,i] = reig[:NE,i] - meanr[i,0], reig[NE:,i]-meanr[i,1]
                noisel[:NE,i], noisel[NE:,i] = leig[:NE,i] - meanl[i,0], leig[NE:,i]-meanl[i,1]
            # noiser[:NE], noiser[NE:] = reig[:NE,0] - meanr[0], reig[NE:,0]-meanr[1]
            # noisel[:NE], noisel[NE:] = leig[:NE,0] - meanl[0], leig[NE:,0]-meanl[1]
            for ir in range(2):
                for jl in range(2):
                    first_perturb_ov[ktrial,it,ir,jl] = np.squeeze(np.reshape(noiser[:,ir],(1,-1))@np.reshape(noisel[:,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl])#@YS 17Nov change to fit ovP#*np.real(eigeng[jl]) #
                    ### separate E and I population
                    first_perturb_ovP[ktrial,it,ir,jl,0]= np.squeeze(np.reshape(noiser[:NE,ir],(1,-1))@np.reshape(noisel[:NE,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl])
                    first_perturb_ovP[ktrial,it,ir,jl,1]= np.squeeze(np.reshape(noiser[NE:,ir],(1,-1))@np.reshape(noisel[NE:,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl]) ### current eigenvalues are multiplied


In [6]:
# data = list_to_dict(lst=lst, string=stg)
data_name = '/Users/yuxiushao/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/DynsStats_Gauss_2FebPRX_36trials_correction_paradoxicalparams.npz'
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/DynsStats_Gauss_2FebPRX_36trials_correction_paradoxicalparams.npz"
# data_name = "[your data folder]/DynsStats_Gauss_2FebPRX_36trials_correction_paradoxicalparams.npz"
data = np.load(data_name,allow_pickle=True)
firing_rateeq = data['firing_rateeq']
lowrank_eq    = data['lowrank_eq']
lowrank_eq_num = data['lowrank_eq_num'] 
contributions_lr = data['contributions_lr'] ### trial, tau, rank, pop
contributions_lr_num = data['contributions_lr_num']

In [7]:
data['params']

array({'gaverage': 0.1, 'gamma': 0.25, 'NE': 800, 'NI': 200, 'tau_series': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])},
      dtype=object)

In [8]:
data_name = '/Users/yuxiushao/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/ConnStats_Gauss_2FebPRX_36trials_correction_paradparams.npz'
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/ConnStats_Gauss_2FebPRX_36trials_correction_paradparams.npz"
# data_name = "[your data folder]/ConnStats_Gauss_2FebPRX_36trials_correction_paradparams.npz"
# np.savez(data_name, **data)
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_ov_series = data['intg_ov_series']
first_perturb_ov = data['first_perturb_ov']
first_perturb_ovP = data['first_perturb_ovP']
intg_mean_series = data['intg_mean_series']
intg_std_series = data['intg_std_series']
intg_std_num_series = data['intg_std_num_series']
mean_shift_ov = data['mean_shift_ov']
intg_crossov_series = data['intg_crossov_series']
intg_crossovPop_series = data['intg_crossovPop_series']
params

array({'gaverage': 0.1, 'gamma': 0.25, 'NE': 800, 'NI': 200, 'tau_series': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])},
      dtype=object)

In [164]:
### compute the theoretical eigenvalue outliers and the spectral radius
### recording 
gaverage = g_max
eigvchn_theo = np.zeros((ntau,2))
radius_theo_map  = np.zeros(ntau)
        
for it, tau in enumerate(tau_series):
    ### solve the equations
    sigmae, sigmai = gaverage, gaverage
    
    x0 = np.array([0.01,0.01])
    sol = fsolve(compute_AdjacencyParams,x0,args=(JE,JI,sigmae,sigmai,Nparams))
    ce,ci = sol[0],sol[1]
    J = JE/NE/ce
    g = JI/NI/ci/J
    # print('ce,ci:',ce,ci)
    # print('J,g:',J,g)
    ### generate the corresponding sparse matrix
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    tau_rec_vec = np.array([[2*tau,-2*tau],[-2*tau,2*tau]])
    # tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,-tau],[-tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
    ### then the theoretical solutions for the eigenvalue outliers
    ### theoretical solutions
    x0 = np.mean(eigvchn_series[:,it,:2].real,axis=0)
    N = int(NE+NI)
    for ir in range(2):
        t1 = fsolve(equations,x0[ir].real,args=[gaverage,tau,(JE-JI),N])
        eigvchn_theo[it,ir] = t1


d:\UseApp\Anaconda\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Compute the mean

In [165]:
### compute the population mean and variance
firing_rate0_mean,firing_rate0_var = np.zeros(2),np.zeros(2)
firing_rate0_mean[0],firing_rate0_mean[1] = np.mean(firing_rate0[:NE]),np.mean(firing_rate0[NE:])
firing_rate0_var[0],firing_rate0_var[1]   = np.var(firing_rate0[:NE]),np.var(firing_rate0[NE:])

# ntau = 6
firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)
firing_rateeq_var[:,:,0],firing_rateeq_var[:,:,1]   = np.var(firing_rateeq[:,:,:NE],axis=2),np.var(firing_rateeq[:,:,NE:],axis=2)

lowrank_eq_num_mean,lowrank_eq_num_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
lowrank_eq_num_mean[:,:,0],lowrank_eq_num_mean[:,:,1] = np.mean(lowrank_eq_num[:,:,:NE],axis=2),np.mean(lowrank_eq_num[:,:,NE:],axis=2)
lowrank_eq_num_var[:,:,0],lowrank_eq_num_var[:,:,1]   = np.var(lowrank_eq_num[:,:,:NE],axis=2),np.var(lowrank_eq_num[:,:,NE:],axis=2)

Filtering the dynamics

In [166]:
#### filter the firing rate
thresh_low = 100.0#0.5#10.0
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau): 
            ### keep the better match solutions
            if np.abs(lowrank_eq_num_mean[ktrial,it,0])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                # print(ktrial,it,0)
            if np.abs(lowrank_eq_num_mean[ktrial,it,1])>thresh_low:
                # print(ktrial,it,1)
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
            if np.abs(lowrank_eq[ktrial,it,0])>thresh_low:
                # print(ktrial,it,0)
                lowrank_eq[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq[ktrial,it,1])>thresh_low:
                # print(ktrial,it,1)
                lowrank_eq[ktrial,it,1] = np.nan
                
#### same for the contribution from different terms
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau):
            ### keep the better match solutions
            if np.abs(contributions_lr[ktrial,it,irr,0])>thresh_low:
                contributions_lr[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr[ktrial,it,irr,1])>thresh_low:
                contributions_lr[ktrial,it,irr,1] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,0])>thresh_low:
                contributions_lr_num[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,1])>thresh_low:
                contributions_lr_num[ktrial,it,irr,1] = np.nan

cuts = 3
### the biggest three and smallest  
for it in range(ntau):
    idxsort = np.argsort(lowrank_eq[:,it,0].real)
    idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
    lowrank_eq[idxnan,it,0] = np.nan     
    ### for 1 
    idxsort = np.argsort(lowrank_eq[:,it,1].real)
    idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
    lowrank_eq[idxnan,it,1] = np.nan
    
for irr in range(2):
    for it in range(0,ntau):
        idxsort = np.argsort(lowrank_eq[:,it,irr].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        lowrank_eq_num[idxnan,it,irr] = np.nan
        idxsort = np.argsort(contributions_lr[:,it,irr,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        contributions_lr[idxnan,it,irr,0] = np.nan
        
        idxsort = np.argsort(contributions_lr[:,it,irr,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        contributions_lr[idxnan,it,irr,1] = np.nan           


Compare the dynamics full-rank (firing_rateeq_mean) with low-rank numerical (lowrank_eq_num) and low-rank theoretical (lowrank_eq)

In [167]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [31 28 18 11  6 24 34 12 14 22 35  7 25  8 20 17 10  0 19  5 33 13 29  1
 26  2 16  4 32 23] 30


In [168]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']

mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
thl = 10
ths = 0.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 3
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        

In [169]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,0],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$E$')
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),color='black',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,0],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),color='gray',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population

# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,1],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),color='orange',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,1],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_num_series[idx_eff,:,1,1]*eigvchn_series[idx_eff,:,1],axis=0),color='green',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$I$')
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-0.4,0.02)
# ax[1].set_yticks([-0.4,0.])
ax[1].set_ylim(-0.35,0.02)
ax[1].set_yticks([-0.35,0.])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5398: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[0] = start
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5399: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N + 1] = end
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5402: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5404: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N+2:, 1] = dep2slice[::-1]
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [170]:
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=1.0)
# for i in range(trials):
#     ax[0].scatter(tau_series,mean_reigvec_num_series[i,:,0,0])
#     print(mean_reigvec_num_series[i,:,0,0])
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
## second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1,0.2])
ax[0].set_xlim(0,tau_max+0.02/8)
ax[0].set_ylim(-0.01,0.04)
ax[0].set_yticks([0,0.04])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)

### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1,0.2])
ax[1].set_xlim(0-dtau/8.0,tau_max+dtau/8)
ax[1].set_ylim(-0.01,0.04)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
fig.tight_layout()

In [182]:
### compute low-rank theoretical
contribution_theo = np.zeros((ntau,2,2))
for it in range(ntau):
    for ir in range(2):
        contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))
        contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))
firing_rate_lwtheo = np.ones((ntau,2))*Inp[0]
firing_rate_lwtheo +=np.sum(contribution_theo.copy(),axis=1)

In [183]:
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
# ax.plot(tau_series, firing_rate0_mean[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
ax.plot(tau_series, firing_rate0_mean[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
### start with the excitatory neuron population
ax.plot(tau_series, np.nanmean(firing_rateeq_mean[idx_eff,:,0],axis=0),marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
# ax.set_ylim(-0.02,0.08)
# ax.set_yticks([0,0.05])
ax.set_ylim(-0.02,0.06)
ax.set_yticks([0,0.05])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)

In [184]:
### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### contribution from the first rank-1 component
ax.plot(tau_series, contribution_theo[:,0,0],marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
ax.plot(tau_series,contribution_theo[:,1,0],marker='o',linestyle='-',color='gray',alpha=1,lw=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.06)
ax.set_yticks([0,0.05])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)

In [181]:
params = {'gaverage':gaverage,
          'gamma':gamma,
          'JE':JE,
          'JI':JI,
          'NE':NE,
          'NI':NI,
          'tau_series':tau_series,
          }
lst = [eigvchn_series, eigrvec_series, eiglvec_series,
       eigrvec_series_rec, eiglvec_series_rec,
        eiglvec0_series, norml0_series, params,
        lowrank_eq, firing_rateeq,lowrank_eq_num,
        ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
        contributions_lr, contributions_lr_num,
        intg_ov_series,first_perturb_ov,first_perturb_ovP,
        intg_mean_series,intg_std_series,intg_std_num_series,mean_shift_ov,
        intg_crossov_series,intg_crossovPop_series,leig0mean_series]
stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
       "eigrvec_series_rec, eiglvec_series_rec,"
        "eiglvec0_series, norml0_series, params,"
        "lowrank_eq, firing_rateeq,lowrank_eq_num,"
        "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
        "contributions_lr, contributions_lr_num,"
        "intg_ov_series,first_perturb_ov,first_perturb_ovP,"
        "intg_mean_series,intg_std_series,intg_std_num_series,mean_shift_ov,"
        "intg_crossov_series,intg_crossovPop_series,leig0mean_series"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/DynsStats_Gauss_13Mar_N1000.npz"
np.savez(data_name, **data)